<a href="https://colab.research.google.com/github/19990715/BART-/blob/main/subtract_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 8.7 MB/s 
     |████████████████████████████████| 1.1 MB 47.2 MB/s 
     |████████████████████████████████| 140 kB 66.7 MB/s 
     |████████████████████████████████| 212 kB 68.5 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
     |████████████████████████████████| 596 kB 56.9 MB/s 
     |████████████████████████████████| 127 kB 55.3 MB/s 
     |████████████████████████████████| 271 kB 56.6 MB/s 
     |████████████████████████████████| 144 kB 50.9 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's depen

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 8.3 MB/s 
     |████████████████████████████████| 6.6 MB 46.6 MB/s 


In [ ]:
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#prepare the dataset
#trainscale和里面的参数
from datasets import Dataset

class Data:
    def __init__(self, data_path, tokenizer):
        self.path = data_path
        self.max_input_length = 1024
        self.max_target_length = 150
        # self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model_path)
        self.tokenizer = tokenizer
        
    def preprocess(self, train_scale=0.8):
        with open(self.path,'r') as f:
            raw_data = f.readlines()
        print(f"=======data_len: {len(raw_data)}")
        start = int(len(raw_data)*train_scale)
        print(f"======train_len: {start}")
        
        raw_train_data = raw_data[:start]
        raw_test_data = raw_data[start:]
        raw_train_test_data = {'train':{'id':[],'document':[],'summary':[]}, \
                               'test':{'id':[],'document':[],'summary':[]}}        
        
        for i,item in enumerate(raw_train_data):
            if len(item.split('\t')) != 3:
                continue
            url,text,label = item.split('\t')
            raw_train_test_data['train']['id'].append(i)
            
            # document is the train data, summary is the substract label
            raw_train_test_data['train']['summary'].append(label.strip())
            raw_train_test_data['train']['document'].append(text.strip())

        for j,item in enumerate(raw_test_data):
            if len(item.split('\t')) != 3:
                continue
            url,text,label = item.split('\t')
            raw_train_test_data['test']['id'].append(i+j+1)
            raw_train_test_data['test']['summary'].append(label.strip())
            raw_train_test_data['test']['document'].append(text.strip())
        
        def preprocess_function(examples):
            
            inputs = examples['document']
            model_inputs = self.tokenizer(inputs, max_length = self.max_input_length, padding = 'max_length', truncation=True)
            
            with self.tokenizer.as_target_tokenizer():
                labels = self.tokenizer(examples['summary'], max_length = self.max_target_length, padding = 'max_length', truncation = True)
            model_inputs['labels'] = labels['input_ids']
            return model_inputs
        
        train_dataset = Dataset.from_dict(raw_train_test_data['train'])
        test_dataset = Dataset.from_dict(raw_train_test_data['test'])
        tokenized_train_dataset = train_dataset.map(preprocess_function)
        tokenized_test_dataset = test_dataset.map(preprocess_function)
        return tokenized_train_dataset, tokenized_test_dataset

In [ ]:
#load the model
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import BartForConditionalGeneration
from transformers import AutoTokenizer

#checkpoint = "distilbart-xsum-9-6"
checkpoint = "sshleifer/distilbart-xsum-9-6"
model = BartForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


Downloading:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [ ]:
#evaluating metric use the rouge_score
import numpy as np
from rouge_score import rouge_scorer, scoring
import nltk

nltk.download('punkt')
  


def compute(predictions, references, rouge_types=None, use_agregator=True, use_stemmer=False):
    if rouge_types is None:
        rouge_types = ["rouge1", "rougeL"]

    scorer = rouge_scorer.RougeScorer(rouge_types=rouge_types, use_stemmer=True)
    if use_agregator:
        aggregator = scoring.BootstrapAggregator()
    else:
        scores = []

    for ref, pred in zip(references, predictions):
        score = scorer.score(ref, pred)
        if use_agregator:
            aggregator.add_scores(score)
        else:
            scores.append(score)

    if use_agregator:
        result = aggregator.aggregate()
    else:
        result = {}
        for key in scores[0]:
            result[key] = list(score[key] for score in scores)

    return result


#metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    #prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    #result["gen_len"] = np.mean(prediction_lens)
    
    print("1")

    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#Configure Hyperparameters
batch_size = 1
args = Seq2SeqTrainingArguments( \
    "/Users/fusi/Desktop/ds/individual_project/demo/model", \
    evaluation_strategy = 'steps', \
    learning_rate = 5e-5, \
    per_device_train_batch_size = batch_size, \
    per_device_eval_batch_size = batch_size, \
    weight_decay = 0.01, \
    save_steps = 5, \
    #save_total_limit = 5, \
    num_train_epochs = 3, \
    predict_with_generate = True, \
    eval_steps = 5, \
    logging_first_step=True, \
    report_to="none")

PyTorch: setting up devices


In [ ]:
#use transformers api to train the data
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model, padding=True)
data = Data('s2.txt', tokenizer)
tokenized_train_dataset, tokenized_test_dataset = data.preprocess()
trainer =Seq2SeqTrainer( \
                       model, \
                       args, \
                       train_dataset = tokenized_train_dataset, \
                       eval_dataset = tokenized_test_dataset, \
                       data_collator = data_collator, \
                       tokenizer = tokenizer, \
                       compute_metrics = compute_metrics)

=======data_len: 30575
======train_len: 24460


  0%|          | 0/5 [00:00<?, ?ex/s]

  0%|          | 0/2 [00:00<?, ?ex/s]

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: document, id, summary. If document, id, summary are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 15


Step,Training Loss,Validation Loss,Rouge1,Rougel
5,7.153100,4.411685,18.149600,13.498500
10,7.153100,2.712535,20.172700,12.009400
15,7.153100,2.242748,20.814800,11.555600


The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: document, id, summary. If document, id, summary are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2
  Batch size = 1


1


Saving model checkpoint to /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-5
Configuration saved in /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-5/config.json
Model weights saved in /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-5/pytorch_model.bin
tokenizer config file saved in /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-5/tokenizer_config.json
Special tokens file saved in /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-5/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: document, id, summary. If document, id, summary are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2
  Batch size = 1


1


Saving model checkpoint to /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-10
Configuration saved in /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-10/config.json
Model weights saved in /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-10/pytorch_model.bin
tokenizer config file saved in /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-10/tokenizer_config.json
Special tokens file saved in /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-10/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: document, id, summary. If document, id, summary are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2
  Batch size = 1


1


Saving model checkpoint to /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-15
Configuration saved in /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-15/config.json
Model weights saved in /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-15/pytorch_model.bin
tokenizer config file saved in /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-15/tokenizer_config.json
Special tokens file saved in /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-15/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=15, training_loss=4.140750821431478, metrics={'train_runtime': 450.0103, 'train_samples_per_second': 0.033, 'train_steps_per_second': 0.033, 'total_flos': 19736077271040.0, 'train_loss': 4.140750821431478, 'epoch': 3.0})

In [ ]:

def predict(sentence):
    inputs = tokenizer([sentence],max_length = 1024, return_tensors='pt')
    summary_ids = model.generate(inputs['input_ids'], num_beams=70, max_length=500,min_length=50,early_stopping=True)
    summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    return ' '.join(summary)

In [ ]:
print(predict("Stocks scored a win in the trading week ended March 27-- as in the best market week in 82 years, even as Friday of that week was a down day. The rally had some investors wondering if 'the low was in.' But in past big selloffs, the confirming low re-test has come months later."))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Stocks in the US have seen their biggest one-day rise in more than a decade, despite fears of a global sell-off in the wake of the Federal Reserve's decision to raise interest rates and raise the prospect of a rate rise.


In [ ]:
print(predict('HNRC will form a Special Purpose Acquisition Corporation with cash and securities for the acquisition of interests in Cunningham Energy with the purpose of acquiring, exploring, and producing oil and gas in the United States and Canada. The proposed investment is for up to USD $100,000,000 in cash and securities and subject to the competition of due diligence and financing. The parties are committed to closing the transaction on or before June 30, 2022.  Under the Letter of Intent additional future development funding for up to $600,000,000 from SPAC and PIPE follow on transactions are also planned.'))

 The Highlands and Islands Oil and Gas Company (HNRC) has announced plans to acquire a Canadian oil and gas exploration and development company in a deal worth up to $2bn (Â£1.6bn) in cash and securities.


In [ ]:
l = []
pre = [] 
with open('summary.txt','r') as f :
  for line in f.readlines():
    line = line.strip('\n')
    l.append(line)
print(len(l))

178


In [ ]:
#print(predict(l[0]))
for i in l:
  p = predict(i)
  pre.append(p)

In [ ]:
df.to_csv('sum.csv')

In [ ]:
from google.colab import files
files.download('sum.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#print(pre)
import pandas as pd
df = pd.DataFrame()
df['summary'] = l
df['subtract'] = pre
df

,summary,subtract
0,"Stocks rose on Friday, finishing off a strong ...",US stocks ended the week in positive territor...
1,"Apple manufactures Smartphones, tablets, PCs, ...","Apple, one of the world's biggest technology c..."
2,Sustainable Impact Investing is gaining tracti...,"The world's biggest fund manager, BlackRock, ..."
3,Stocks were mixed on Friday morning as investo...,(Close): Wall Street markets closed mixed on ...
4,"Innovation may be hard to define but, to borro...",As the world mourns the loss of hundreds of th...
...,...,...
173,,A look back at some of the key moments that yo...
174,,A look back at some of the key moments that yo...
175,,A look back at some of the key moments that yo...
176,,A look back at some of the key moments that yo...
